In [1]:
%%HTML 
<style>.container{width:100%;}</style>

Dieses Notebook hängt vom Notebook `Splay-Trees` ab.

In [2]:
%run Splay-Trees.ipynb

# Mengen auf Basis von Splay Trees

Wir benutzen nun die Splay Trees, um Mengen (`set`) in Python mit Ordnung zu unterstützen.

Dazu bauen wir die hier gelisteten Methoden nach ([Permalink zur Datei, aus der dies generiert wird](https://github.com/python/cpython/blob/3.7/Objects/setobject.c "R. D. Hettinger et al. (2019): cpython/Objects/setobject.c, GitHub")):

In [3]:
help(set)

Help on class set in module builtins:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iand__(self, value, /)
 |      Return self&=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ixor__(self, value, /)
 |      Re

Wir definieren dafür die Klasse `OrderedSet`.

In [4]:
class OrderedSet():
    pass

## Konstruktion

Wenn wir das Konstruieren eines Sets aus einem `iterable` unterstützen wollen, begegnen wir dem Problem der Implementierung von Mengen modifizierbarer Objekte, zum Beispiel Mengen von Mengen.

Sei $N$ eine Menge und $M$ ein modifizierbares Objekt, zum Beispiel eine Menge, und sei $M \in N$. Wie stellen wir sicher, dass $N$ geordnet bleibt, wenn $M$ geändert wird? Wir könnten verlangen, dass in $N$ ein Element, wann immer es preisgegeben wird, neu einsortiert werden muss. Abgesehen vom damit verbundenen Berechnungsaufwand ist dies aber nicht praktikabel, da auf $M$ auch Referenzen gehalten werden können, die nicht von $N$ abhängig sind, und somit $M$ geändert werden könnte, ohne dass $N$ davon erfährt. (Die $N$-seitige Überwachung aller gehaltenen Objekte wäre noch unpraktischer als das erneute Einsortieren.)

Ein vergleichbares Problem ergibt sich übrigens auch bei den Hashtabellen in der Referenzimplementierung CPython. Hier kann keine Ordnungsbedingung verletzt werden, jedoch ist im Allgemeinen ein Objekt nach Änderung nicht mehr unter ihrem vorigen Hash zu finden. Der Python-Standard gibt daher vor, dass es strenggenommen keine Mengen von Mengen gibt – [es gibt nur Mengen von unmodifizierbaren Mengen](https://docs.python.org/3.7/library/stdtypes.html#set "Python Software Foundation (2019): The Python Standard Library/Built-in Types/set, Python Documentation"), den `frozenset`s. Ebenso gibt es beispielsweise keine Mengen von Listen.

Wir wollen auch die `frozenset`s geordnet haben und werden sie parallel mitimplementieren.

In [5]:
class OrderedFrozenset():
    pass

Wir übernehmen `_arb_lt`, `_arb_eq` und `_arb_gt` von `Node`, sodass wir direkt Zugriff darauf haben.

In [6]:
OrderedSet._arb_lt = OrderedFrozenset._arb_lt = Node._arb_lt
OrderedSet._arb_eq = OrderedFrozenset._arb_eq = Node._arb_eq
OrderedSet._arb_gt = OrderedFrozenset._arb_gt = Node._arb_gt

Konstruieren wir nun eine Menge aus einem `iterable`, so fügen wir die Elemente einzeln ein. Wir arbeiten direkt mit `Node`s, wrappen also nicht noch zusätzlich einen `SplayTree`. Wir überprüfen dabei die hinzuzufügenden Elemente darauf, ob sie in einer Menge existieren dürfen, indem wir überprüfen, ob sie eine `__hash__`-Methode haben.

In [7]:
def __init__(self, iterable=[]):
    if isinstance(self, OrderedFrozenset) and hasattr(self, "_tree"):
        return  # frozenset was already created
    self._tree = None
    iterator = iter(iterable)
    try:
        element = next(iterator)
        if not element.__hash__:
            raise TypeError(f"unhashable type: '{type(element).__name__}'")
        self._tree = Node(element, None, None)
        while True:
            element = next(iterator)
            if not element.__hash__:
                raise TypeError("unhashable type: " +
                                f"'{type(element).__name__}'")
            self._tree = self._tree.insert(element)
    except StopIteration:
        pass

OrderedSet.__init__ = OrderedFrozenset.__init__ = __init__
del __init__

### Lexikographische Ordnung

Wir müssen, wenn wir geordnete Mengen von geordneten Mengen unterstützen wollen, die totalen Ordnungen `<` und `>` sowie die Operation `==` zwischen geordneten Mengen definieren – diese Operatoren werden von `_arb_lt` und verwandten Methoden und somit mittelbar von `splay` benutzt. Wir müssen dabei leider das Verhalten von `set` und `frozenset` verändern; für diese drücken `<` und `>` die echte Ober- bzw. Teilmenge aus. Alternativ könnten wir in `_arb_lt` und verwandten Methoden vorher den Typ überprüfen und im Fall von `OrderedSet` und `OrderedFrozenset` nicht `<` und `>`, sondern andere Methoden verwenden. Die damit verbundenen Performanceeinbußen sind jedoch nicht vertretbar.

Eine totale Ordnung, die sich anbietet, ist die lexikographische Ordnung. Hier wird durch die Mengen iteriert und das erste ungleiche Wertepaar entscheidet die Ungleichheit. Wir definieren dafür zunächst die Iteration `__iter__`, um durch die Elemente in der Menge iterieren zu können.

Dies erreichen wir, indem wir den Baum wie bei einer *Inorder*-Ausgabe durchlaufen, wobei wir zur Ausgabe das Keyword `yield` benutzen. Die Implementierung ist iterativ und hält sich einen eigenen Stapel, da dieser in seiner Größe weniger begrenzt ist als der Aufrufstapel, der sich bei einer rekursiven Implementierung aufbauen würde. Für diesen Stapel benutzen wir [`deque`](https://docs.python.org/3.7/library/collections.html#collections.deque "Python Software Foundation (2019): The Python Standard Library/Data Types/collections/deque, Python Documentation") (double-ended queue) aus den `collections` der Standard Library, da sich diese für größere Stapel besser eignet als die Liste.

In [8]:
def __iter__(self):
    stack = deque()
    tree = self._tree
    while stack or tree is not None:
        if tree is not None:
            stack.append(tree)
            tree = tree.left
            continue
        tree = stack.pop()
        yield tree.payload
        tree = tree.right

OrderedSet.__iter__ = OrderedFrozenset.__iter__ = __iter__
del __iter__

Nun können wir (für `<`) die Methode `__lt__` mit lexikographischer Ordnung definieren. Wir werfen bei Typinkompatibilität den `TypeError`, mit dem `_arb_lt` und verwandte Methoden bei Typinkompatibilitäten umgehen.

In [9]:
def __lt__(self, other):
    if (not isinstance(other, OrderedSet)
            and not isinstance(other, OrderedFrozenset)):
        raise TypeError("'<' not supported between instances of " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")
    x_iter, y_iter = iter(self), iter(other)
    while True:
        try:
            y_item = next(y_iter)
        except StopIteration:
            return False  # x is longer or equal
        try:
            x_item = next(x_iter)
        except StopIteration:
            return True  # x is shorter
        if self._arb_lt(x_item, y_item):
            return True
        if self._arb_gt(x_item, y_item):
            return False

OrderedSet.__lt__ = OrderedFrozenset.__lt__ = __lt__
del __lt__

Ähnlich definieren wir `__gt__` für `>`.

In [10]:
def __gt__(self, other):
    if (not isinstance(other, OrderedSet)
            and not isinstance(other, OrderedFrozenset)):
        raise TypeError("'>' not supported between instances of " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")
    x_iter, y_iter = iter(self), iter(other)
    while True:
        try:
            x_item = next(x_iter)
        except StopIteration:
            return False  # x is shorter or equal
        try:
            y_item = next(y_iter)
        except StopIteration:
            return True  # x is longer
        if self._arb_gt(x_item, y_item):
            return True
        if self._arb_lt(x_item, y_item):
            return False

OrderedSet.__gt__ = OrderedFrozenset.__gt__ = __gt__
del __gt__

Für `__eq__` (`==`) müssen wir nur Element für Element vergleichen. Wir verändern hier auch nicht das Verhalten im Vergleich zu den Vorbildern `set` und `frozenset`. Hier ist  der Vergleich mit fremden Typen kein `TypeError`, sondern einfach die Ungleichheit. 

In [11]:
def __eq__(self, other):
    if (not isinstance(other, OrderedSet)
            and not isinstance(other, OrderedFrozenset)):
        return False
    x_iter, y_iter = iter(self), iter(other)
    while True:
        try:
            x_item = next(x_iter)
        except StopIteration:
            try:  # assert y is also exhausted
                next(y_iter)
            except StopIteration:
                return True
            return False
        try:
            y_item = next(y_iter)
        except StopIteration:
            return False  # x was not exhausted
        if not self._arb_eq(x_item, y_item):
            return False

OrderedSet.__eq__ = OrderedFrozenset.__eq__ = __eq__
del __eq__

Wir definieren an dieser Stelle noch `__ne__` (`!=`), bei dem wir nur das Ergebnis von `==` invertieren.

In [12]:
def __ne__(self, other):
    return not self == other

OrderedSet.__ne__ = OrderedFrozenset.__ne__ = __ne__
del __ne__

### Hashen

Der letzte Schritt, um `OrderedFrozenset`s in `OrderedSet`s haben zu können, ist die Unterstützung einer `__hash__`-Funktion für `OrderedFrozenset` (`OrderedSet` ist wegen seiner Veränderlichkeit bewusst **nicht** hashbar). Wir addieren dazu die Hashwerte aller Elemente, deren Hashbarkeit wir ja schon zur Zeit des Hinzufügens überprüft haben.

In [13]:
def __hash__(self):
    return sum(hash(element) for element in self)

OrderedFrozenset.__hash__ = __hash__
del __hash__

Um das Verhalten von Python abzubilden, werfen wir im Fall von `OrderedSet` einen `TypeError`, wenn versucht wird, eines zu hashen.

In [14]:
def __hash__(self):
    raise TypeError(f"unhashable type: '{type(self).__name__}'")

OrderedSet.__hash__ = __hash__
del __hash__

## Operationen mit einem Element

Wir implementieren im Folgenden die Methoden von `set` und `frozenset` für `OrderedSet` und `OrderedFrozenset`. Wir beginnen mit den Operationen mit einem einzelnen Element als Parameter.

### Hinzufügen

Wir implementieren `add`, das Hinzufügen eines Elements, ähnlich wie beim `SplayTree`:

In [15]:
def add(self, element):
    if not element.__hash__:
        raise TypeError(f"unhashable type: '{type(element).__name__}'")
    if self._tree is None:
        self._tree = Node(element, None, None)
    else:
        self._tree = self._tree.insert(element)

OrderedSet.add = add
del add

### Entfernen

Das Gegenstück zu `add` ist `remove`, das Entfernen eines Elementes.

In [16]:
def remove(self, element):
    if not element.__hash__ and not (isinstance(element, OrderedSet)
                                     or isinstance(element, OrderedFrozenset)):
        raise TypeError(f"unhashable type: '{type(element).__name__}'")
    if self._tree is None:
        raise KeyError(element)
    rc, self._tree = self._tree.remove(element)
    if not rc:
        raise KeyError(element)

OrderedSet.remove = remove
del remove

Wir implementieren auch `discard`, welches ähnlich zu `remove` ist, aber keinen Fehler verursacht, falls das Element nicht vorhanden ist.

In [17]:
def discard(self, element):
    if not element.__hash__ and not (isinstance(element, OrderedSet)
                                     or isinstance(element, OrderedFrozenset)):
        raise TypeError(f"unhashable type: '{type(element).__name__}'")
    if self._tree is not None:
        _, self._tree = self._tree.remove(element)

OrderedSet.discard = discard
del discard

`add`, `remove` und `discard` sind **keine** Methoden von `OrderedFrozenset`, weil diese Operationen die Menge modifizieren!

### Finden

Eine nichtmodifizierende Operation mit einem Element als Parameter ist `__contains__` (`in`), die wir ebenfalls für `Node` schon implementiert haben.

In [18]:
def __contains__(self, element):
    if not element.__hash__ and not (isinstance(element, OrderedSet)
                                     or isinstance(element, OrderedFrozenset)):
        raise TypeError(f"unhashable type: '{type(element).__name__}'")
    if self._tree is None:
        return False
    contains, self._tree = self._tree.contains(element)
    return contains

OrderedSet.__contains__ = OrderedFrozenset.__contains__ = __contains__
del __contains__

## Operationen ohne Parameter

Wir implementieren im Folgenden Operationen auf die geordnete Menge, die überhaupt keine Parameter haben. Wir beginnen wieder mit solchen, die die Menge verändern.

### Kopieren

Wir implementieren `copy`, welches eine Kopie der Menge zurückgibt.

In [19]:
def copy(self):
    if isinstance(self, OrderedSet):
        return OrderedSet(list(self))
    return OrderedFrozenset(list(self))

OrderedSet.copy = OrderedFrozenset.copy = copy
del copy

### Entfernen eines beliebigen Elements

Wir implementieren `pop`, welches ein beliebiges Element entfernt und zurückgibt. Wir entfernen einfach die Wurzel des Baums.

In [18]:
def pop(self):
    if self._tree is None:
        raise KeyError("pop from an empty set")
    popped = self._tree.payload
    self.remove(popped)
    return popped

OrderedSet.pop = pop
del pop

### Leeren

Wir implementieren auch `clear`, welches alle Elemente aus der Menge entfernt. Wir setzen einfach den Baum auf `None`.

In [19]:
def clear(self):
    self._tree = None

OrderedSet.clear = clear
del clear

### Länge

Wir kehren zurück zu den Operationen, die auch für `OrderedFrozenset` erlaubt sind, und implementieren `__len__`, welches die Länge von Mengen berechnet und dann mit `len(some_ordered_set)` benutzt werden kann.

In [20]:
def __len__(self):
    length = 0
    for el in self:
        length += 1
    return length

OrderedSet.__len__ = OrderedFrozenset.__len__ = __len__
del __len__

### Darstellung

Wir implementieren auch `__repr__`, welches eine String-Darstellung anbietet. Wir wollen zum Beispiel für die Elemente `a, b` die Darstellung `OrderedSet([a, b])` haben und implementieren:

In [21]:
def __repr__(self):
    return f"{type(self).__name__}({list(self)})"

OrderedSet.__repr__ = OrderedFrozenset.__repr__ = __repr__
del __repr__

### Pickling

`set` implementiert `__reduce__`, welches von der *pickle*-API aufgerufen wird. Diese API dient zur Serialisierung. Wir implementieren für diese API nicht `__reduce__`, sondern das simplere [`__getnewargs__`](https://docs.python.org/3/library/pickle.html#object.__getnewargs__ "Python Software Foundation (2020): The Python Standard Library/Data Persistence/object.__getnewargs__(), Python Documentation"), welches die Argumente, mit denen eine Rekonstruktion erreicht wird, zurückgibt. Wir geben den Mengeninhalt als Simpel aus einer Liste aus, sodass zum Beispiel für die Elemente `a, b` beim Deserialisieren  `OrderedSet([a, b],)` aufgerufen wird.

In [22]:
def __getnewargs__(self):
    return (list(self),)

OrderedSet.__getnewargs__ = OrderedFrozenset.__getnewargs__ = __getnewargs__
del __getnewargs__

### Minimum und Maximum

Wir implementieren noch die *zusätzliche* Methode `minimum`. Mit `__iter__` funktioniert zwar bereits das Built-in `min`, wir können dies mit einem Baum aber effizienter gestalten: Wir geben die Nutzlast des linkesten Knotens zurück. Für `Node` definieren wir

In [23]:
def minimum(self):
    while self.left is not None:
        self = self.left
    return self.payload

Node.minimum = minimum
del minimum

und für `OrderedSet` und `OrderedFrozenset`

In [24]:
def minimum(self):
    if self._tree is None:
        raise ValueError("Set is empty")
    return self._tree.minimum()

OrderedSet.minimum = OrderedFrozenset.minimum = minimum
del minimum

Wir implementieren auch `maximum`, bei der wir die Nutzlast des rechtesten Knotens zurückgeben.

In [25]:
def maximum(self):
    while self.right is not None:
        self = self.right
    return self.payload

Node.maximum = maximum
del maximum

und für die `Set`-Klassen

In [26]:
def maximum(self):
    if self._tree is None:
        raise ValueError("Set is empty")
    return self._tree.maximum()

OrderedSet.maximum = OrderedFrozenset.maximum = maximum
del maximum

## Boolesche Operationen auf andere Mengen

Wir implementieren im Folgenden Operationen auf andere Mengen, die einen Wahrheitswert als Ergebnis haben. `==` und `!=` hatten wir schon für die lexikographische Ordnung implementiert.

### Teilmenge

Wir haben `issubset`, `__le__` (`<=`, in der Mengenlehre $\subseteq$) und `__lt__` (`<`, in der Mengenlehre $\subset$ bzw. $\subsetneq$) zu implementieren. `issubset` arbeitet wie `__le__`, erlaubt aber einen beliebigen Parameter als Operand, statt nur Mengen. Wir werden die echte Teilmenge nicht unter dem Namen `<` implementieren können, weil wir diesen Operator für die lexikographische Ordnung benutzen. Wir werden dafür in Anlehnung an `issubset` die Methode `is_proper_subset` implementieren, die *keine* Methode von `set` oder `frozenset` ist.

Für die unechte Teilmenge implementieren wir die Hilfsfunktion `_subseteq`, welches von geordneten Mengen $x, y$ die Relation $x \subseteq y$ überprüft. So brauchen wir für die eigentlichen Methoden nur `_subseteq` mit den entsprechenden Parametern aufrufen. Dies erspart besonders für die Obermenge Arbeit.

Hier gehen wir für jedes Element aus $x$ so lange in $y$ weiter, bis die Elemente gleich sind. In diesem Fall gehen wir in beiden Mengen ein Element weiter. Ist ein Element aus $y$ größer, so kam ein Element aus $x$ nicht in $y$ vor, und wir verneinen die Relation $\subseteq$. Ist $x$ erschöpft, so waren alle Elemente in $y$. Ist $y$ erschöpft, so war mindestens ein Element von $x$ nicht in $y$.

In [27]:
def _subseteq(self, x, y):
    x_iter, y_iter = iter(x), iter(y)
    while True:
        try:
            x_item = next(x_iter)
        except StopIteration:
            return True
        while True:
            try:
                y_item = next(y_iter)
            except StopIteration:
                return False
            if self._arb_lt(x_item, y_item):
                return False
            if self._arb_eq(x_item, y_item):
                break

OrderedSet._subseteq = OrderedFrozenset._subseteq = _subseteq
del _subseteq

So können wir `issubset` implementieren, wo wir das andere Iterable gegebenenfalls in ein `OrderedFrozenset` bringen und so sortieren.

In [28]:
def issubset(self, other):
    if type(other) not in (OrderedSet, OrderedFrozenset):
        other = OrderedFrozenset(other)
    return self._subseteq(self, other)

OrderedSet.issubset = OrderedFrozenset.issubset = issubset
del issubset

Bei `__le__` erlauben wir für den anderen Operanden nur `OrderedSet` und `OrderedFrozenset`.

In [31]:
def __le__(self, other):
    if (not isinstance(other, OrderedSet)
            and not isinstance(other, OrderedFrozenset)):
        raise TypeError("'<=' not supported between instances of " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")
    return self._subseteq(self, other)

OrderedSet.__le__ = OrderedFrozenset.__le__ = __le__
del __le__

Für `is_proper_subset` und später `is_proper_superset` definieren wir `_subsetneq`, welches $x \subset y$ überprüft. Hier halten wir uns die auf `False` initialisierte Variable `proper_subset`, mit deren Hilfe wir sicherstellen, dass $x$ eine echte Teilmenge von $y$ ist. Sie wird auf `True` gesetzt, sobald ein Element aus $y$ kleiner ist als aus $x$. Ist $x$ erschöpft und `proper_subset` nicht `True`, so prüfen wir, ob noch Elemente in $y$ sind, dann ist ebenfalls $x \subset y$.

In [30]:
def _subsetneq(self, x, y):
    x_iter, y_iter = iter(x), iter(y)
    proper_subset = False
    while True:
        try:
            x_item = next(x_iter)
        except StopIteration:
            if not proper_subset:
                try:  # assert y is not exhausted
                    next(y_iter)
                except StopIteration:
                    return False
            return True
        while True:
            try:
                y_item = next(y_iter)
            except StopIteration:
                return False
            if self._arb_lt(x_item, y_item):
                return False
            if self._arb_gt(x_item, y_item):
                proper_subset = True
            else:
                break

OrderedSet._subsetneq = OrderedFrozenset._subsetneq = _subsetneq
del _subsetneq

So können wir `is_proper_subset` ähnlich wie `issubset` implementieren.

In [31]:
def is_proper_subset(self, other):
    if type(other) not in (OrderedSet, OrderedFrozenset):
        other = OrderedSet(other)
    return self._subsetneq(self, other)

OrderedSet.is_proper_subset = is_proper_subset
OrderedFrozenset.is_proper_subset = is_proper_subset
del is_proper_subset

### Obermenge

Wir können für die Obermenge jetzt `_subseteq` und `_subsetneq` wiederverwenden und müssen nur die Parameter ändern. Analog zur den Operationen für die Untermenge haben wir `issuperset`…

In [32]:
def issuperset(self, other):
    if type(other) not in (OrderedSet, OrderedFrozenset):
        other = OrderedSet(other)
    return self._subseteq(other, self)

OrderedSet.issuperset = OrderedFrozenset.issuperset = issuperset
del issuperset

…`__ge__` (`>=`, in der Mengenlehre $\supseteq$)…

In [35]:
def __ge__(self, other):
    if (not isinstance(other, OrderedSet)
            and not isinstance(other, OrderedFrozenset)):
        raise TypeError("'>=' not supported between instances of " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")
    return self._subseteq(other, self)

OrderedSet.__ge__ = OrderedFrozenset.__ge__ = __ge__
del __ge__

…und `is_proper_superset` statt `__gt__` (`>`, in der Mengenlehre $\supset$ bzw. $\supsetneq$).

In [34]:
def is_proper_superset(self, other):
    if type(other) not in (OrderedSet, OrderedFrozenset):
        other = OrderedSet(other)
    return self._subsetneq(other, self)

OrderedSet.is_proper_superset = is_proper_superset
OrderedFrozenset.is_proper_superset = is_proper_superset
del is_proper_superset

### Überprüfen, ob Mengen disjunkt sind

Wir implementieren zuletzt `isdisjoint`, das überprüft, ob Mengen disjunkt sind, das heißt, ob sie keine gemeinsamen Elemente haben. Wir iterieren parallel durch die Mengen und überprüfen, dass wir jedes Element in nur einer Menge wiederfinden.

In [35]:
def isdisjoint(self, *args):
    sets = [self] + [OrderedSet(x) if type(x) not in
                     (OrderedSet, OrderedFrozenset) else x for x in args]
    iters = {id(x): iter(x) for x in sets}
    items = dict()
    for k, v in iters.items():
        try:
            items[k] = next(v)
        except StopIteration:
            pass
    while items:
        minimum = None
        min_id = None
        for k, v in items.items():
            if not minimum:
                min_id = k
                minimum = v
                continue
            if self._arb_eq(v, minimum):
                return False
            if self._arb_lt(v, minimum):
                min_id = k
                minimum = v
        try:
            items[min_id] = next(iters[min_id])
        except StopIteration:
            items.pop(min_id)
    return True

OrderedSet.isdisjoint = OrderedFrozenset.isdisjoint = isdisjoint
del isdisjoint

## Verknüpfung von Mengen

Zuletzt implementieren wir die Operationen auf andere Mengen, die eine Menge als Ergebnis haben. Wir beginnen mit den einfacheren Vereinigungs- und Differenzmengen.

### Vereinigungsmenge

Bei Vereinigungsmengen ($\cup$) fügen wir einfach alle Elmente hinzu. Zunächst definieren wir die Methode `union`.

In [36]:
def union(self, *args):
    union = OrderedSet(self)
    for arg in args:
        for el in arg:
            union.add(el)
    if type(self) == OrderedFrozenset:
        return OrderedFrozenset(union)
    return union

OrderedSet.union = OrderedFrozenset.union = union
del union

Wie für `intersection`, welches beliebig viele Iterable erlaubt, und für `__and__`, welches genau eine Menge erlaubt, haben wir jetzt entsprechend zu `union` das `__or__` (`|`).

In [39]:
def __or__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.union(other)
    else:
        raise TypeError("unsupported operand type(s) for |: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__or__ = OrderedFrozenset.__or__ = __or__
del __or__

Das umgekehrte `__ror__` funktioniert wie `__or__`, abgesehen von der Fehlermeldung.

In [40]:
def __ror__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.union(other)
    else:
        raise TypeError("unsupported operand type(s) for |: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__ror__ = OrderedFrozenset.__ror__ = __ror__
del __ror__

*In-place* ist die `union` entsprechende Operation `update`. Hier operieren wir direkt auf `self`. Diese Methode ordnen wir nicht `OrderedFrozenset` zu, da die Menge überschrieben wird.

und `__ior__` (`|=`). 

In [39]:
def update(self, *args):
    for arg in args:
        for el in arg:
            self.add(el)

OrderedSet.update = update
del update

Als Operator haben wir `__ior__` (`|=`).

In [42]:
def __ior__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        self.update(other)
        return self
    else:
        raise TypeError("unsupported operand type(s) for |=: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__ior__ = __ior__
del __ior__

### Differenzmenge

Es folgt die Differenzmenge ($\setminus$), für die wir einfach alle Elemente entfernen. Dafür haben wir `difference`.

In [41]:
def difference(self, *others):
    difference = OrderedSet(self)
    for other in others:
        for el in other:
            if not el.__hash__:
                raise TypeError(f"unhashable type: '{type(el).__name__}'")
            difference.discard(el)
    if type(self) == OrderedFrozenset:
        return OrderedFrozenset(difference)
    return difference

OrderedSet.difference = OrderedFrozenset.difference = difference
del difference

Als Operator haben wir `__sub__`.

In [44]:
def __sub__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.difference(other)
    else:
        raise TypeError("unsupported operand type(s) for -: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__sub__ = OrderedFrozenset.__sub__ = __sub__
del __sub__

`__rsub__` allerdings verhält sich nicht kommutativ, sodass wir auch für den erfolgreichen Teil die Operanden umdrehen.

In [45]:
def __rsub__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return other.difference(self)
    else:
        raise TypeError("unsupported operand type(s) for -: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__rsub__ = OrderedFrozenset.__rsub__ = __rsub__
del __rsub__

In-place haben wir `difference_update`…

In [46]:
def difference_update(self, *others):
    for other in others:
        for el in other:
            if not el.__hash__:
                raise TypeError(f"unhashable type: '{type(el).__name__}'")
            self.discard(el)

OrderedSet.difference_update = difference_update
del difference_update

…und `__isub__` (`-=`).

In [47]:
def __isub__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        self.difference_update(other)
        return self
    else:
        raise TypeError("unsupported operand type(s) for |=: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__isub__ = __isub__
del __isub__

### Schnittmenge

Komplexer sind `__and__` (`&`) und die verwandte `intersection`. Beide Methoden berechnen die Schnittmenge ($\cap$) zweier Mengen, haben jedoch nicht ganz dasselbe Verhalten. `__and__` funktioniert nur mit Mengen, und operiert auf nur eine zusätzliche Menge; hingegen nimmt `intersection` beliebig viele iterable Sammlungen. Wir werden daher `intersection` implementieren und `__and__` mit seinen eigenen Begrenzungen `intersection` aufrufen lassen.

Wir machen aus allen zu schneidenden Sammlungen `OrderedSet`s, falls sie nicht schon welche sind, und iterieren dann simultan durch diese Sammlungen, wobei wir von den simultan betrachteten Elementen immer alle wegwerfen, die kleiner als das Maximum der betrachteten Elemente sind, und genau dann ein Element in unsere neue Menge aufnehmen, wenn alle betrachteten Elemente gleich sind.

Für das Maximum definieren wir `_arb_max`, um im Stil von `_arb_gt` ein Maximum aus beliebigen Elementen berechnen zu können.

In [48]:
def _arb_max(self, other, *others):
    maximum = other
    for el in others:
        if self._arb_gt(el, other):
            maximum = el
    return maximum

OrderedSet._arb_max = OrderedFrozenset._arb_max = _arb_max
del _arb_max

Nun können wir `intersection` definieren.

In [47]:
def intersection(self, *args):
    sets = [self] + [OrderedSet(x) if type(x) not in
                     (OrderedSet, OrderedFrozenset) else x for x in args]
    iters = {id(x): iter(x) for x in sets}
    intersection = OrderedSet()
    try:
        items = {k: next(v) for k, v in iters.items()}
        while True:
            max_of_items = self._arb_max(*items.values())
            all_equal = True
            for k, v in items.items():
                if self._arb_lt(v, max_of_items):
                    all_equal = False
                    items[k] = next(iters[k])
            if all_equal:
                intersection.add(next(iter(items.values())))  # arbitrary item
                items = {k: next(iters[k]) for k in items.keys()}
    except StopIteration:
        pass
    if type(self) == OrderedFrozenset:
        return OrderedFrozenset(intersection)
    return intersection

OrderedSet.intersection = OrderedFrozenset.intersection = intersection
del intersection

Für `__and__` begrenzen wir entsprechend die Eingabe.

In [50]:
def __and__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.intersection(other)
    else:
        raise TypeError("unsupported operand type(s) for &: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")


OrderedSet.__and__ = OrderedFrozenset.__and__ = __and__
del __and__

Wir benötigen auch `__rand__` (reverse `__and__`), bei welchem die Operanden vertauscht sind. Die Operation $\cap$ verhält sich jedoch kommutativ, weshalb in der Implementierung lediglich die Fehlermeldung beim `TypeError` von `__and__` verschieden ist.

In [51]:
def __rand__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.intersection(other)
    else:
        raise TypeError("unsupported operand type(s) for &: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__rand__ = OrderedFrozenset.__rand__ = __rand__
del __rand__

### Schnittmenge in-place

Verwandt mit diesen Methoden sind `__iand__` (`&=`) und `intersection_update`, die die Schnittmenge *in-place* berechnen, das heißt, sie überschreiben die Menge direkt. Hier entfernen wir aus `self` alle Elemente, die nicht in allen übrigen Mengen zu finden sind. Wir iterieren parallel durch `self` und die anderen Mengen und entfernen ein Element aus `self`, sobald es in einer anderen Menge übersprungen wird. Weil der Baum fortlaufend geändert wird, ist `__iter__` für `self` ungeeignet. Wir iterieren daher, indem wir das Minimum splayen, und statt jedem `next` das Minimum aus dem rechten Teilbaum splayen. Somit ist der betrachtete Knoten immer die Wurzel.

Gerade weil diese Methoden die Menge überschreiben, sind sie keine Methoden von `OrderedFrozenset`.

In [50]:
def intersection_update(self, *args):
    if self._tree is None:
        return
    sets = [OrderedSet(x) if type(x) not in
            (OrderedSet, OrderedFrozenset) else x for x in args]
    iters = {id(x): iter(x) for x in sets}
    try:
        self._tree = self._tree._splay(self.minimum())
        items = {k: next(v) for k, v in iters.items()}
        while True:
            if self._tree.right is None:
                return  # self._tree is exhausted
            all_equal = True
            for k, v in items.items():
                if self._tree._arb_lt(v, self._tree.payload):
                    all_equal = False
                    items[k] = next(iters[k])
                elif self._tree._arb_gt(v, self._tree.payload):
                    all_equal = False
                    # next item in tree becomes root after deletion
                    self.remove(self._tree.payload)
                    if self._tree is None:
                        return  # set is empty
                    break  # break out of for loop since we have a new root
            if all_equal:
                # move on to next
                self._tree = self._tree._splay(self._tree.right.minimum())
    except StopIteration:
        # remove all larger than largest kept item
        self._tree.right = None

OrderedSet.intersection_update = intersection_update
del intersection_update

Als Operator haben wir `__iand__`.

In [53]:
def __iand__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        self.intersection_update(other)
        return self
    else:
        raise TypeError("unsupported operand type(s) for &=: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__iand__ = __iand__
del __iand__

### Symmetrische Differenz

Für die symmetrische Differenz ($\bigtriangleup$) brauchen wir out-of-place `symmetric_difference`. Wir iterieren parallel durch die Mengen, berechnen dabei das Minimum unter den gerade betrachteten Elementen und speichern auch, welche Elemente dieses Minimum sind. Wenn dies genau ein Element war, so wird es in die Ergebnismenge aufgenommen, andernfalls gehen wir weiter.

In [52]:
def symmetric_difference(self, *args):
    sets = [self] + [OrderedSet(x) if type(x) not in
                     (OrderedSet, OrderedFrozenset) else x for x in args]
    iters = {id(x): iter(x) for x in sets}
    sdifference = OrderedSet()
    items = dict()
    for k, v in iters.items():
        try:
            items[k] = next(v)
        except StopIteration:
            pass
    while items:
        fwd = []  # items to be forwarded
        for k, v in items.items():
            if not v.__hash__:
                raise TypeError(f"unhashable type: '{type(v).__name__}'")
            if not fwd:
                fwd.append((k, v))
                continue
            if self._arb_eq(v, fwd[0][1]):
                fwd.append((k, v))
                continue
            if self._arb_lt(v, fwd[0][1]):
                fwd.clear()
                fwd.append((k, v))
        if len(fwd) == 1:
            sdifference.add(fwd[0][1])
        for k, _ in fwd:
            try:
                items[k] = next(iters[k])
            except StopIteration:
                items.pop(k)
    if type(self) == OrderedFrozenset:
        return OrderedFrozenset(sdifference)
    return sdifference

OrderedSet.symmetric_difference = symmetric_difference
OrderedFrozenset.symmetric_difference = symmetric_difference
del symmetric_difference

Als Operation haben wir `__xor__` (`^`)…

In [55]:
def __xor__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.symmetric_difference(other)
    else:
        raise TypeError("unsupported operand type(s) for ^: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__xor__ = OrderedFrozenset.__xor__ = __xor__
del __xor__

…und umgekehrt `__rxor__`, wobei `^` sich kommutativ verhält.

In [56]:
def __rxor__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        return self.symmetric_difference(other)
    else:
        raise TypeError("unsupported operand type(s) for ^: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__rxor__ = OrderedFrozenset.__rxor__ = __rxor__
del __rxor__

### Symmetrische Differenz in-place

Wir gehen, wenn wir die symmetrische Differenz in-place, also überschreibend, berechnen, für die Argumente ähnlich wie bei out-of-place vor. Immer, wenn wir aus diesen Mengen einen Kandidaten gefunden haben, gehen wir durch den Baum von `self`, bis wir über den Verbleib des gefundenen Elementes entscheiden.

In [55]:
def symmetric_difference_update(self, *args):
    if self._tree is None:
        self = self.symmetric_difference(args)
    sets = [OrderedSet(x) if type(x) not in
            (OrderedSet, OrderedFrozenset) else x for x in args]
    iters = {id(x): iter(x) for x in sets}
    self._tree = self._tree._splay(self.minimum())
    items = dict()
    for k, v in iters.items():
        try:
            items[k] = next(v)
        except StopIteration:
            pass
    while items:
        fwd = []
        for k, v in items.items():
            if not fwd:
                fwd.append((k, v))
                continue
            if self._tree._arb_eq(v, fwd[0][1]):
                fwd.append((k, v))
                continue
            if self._tree._arb_lt(v, fwd[0][1]):
                fwd.clear()
                fwd.append((k, v))
        if len(fwd) == 1:
            iteration_done = False
            while self._tree._arb_lt(self._tree.payload, fwd[0][1]):
                self._tree = self._tree._splay(self._tree.right.minimum())
                if self._tree.right is None:  # tree is exhausted
                    self.add(fwd[0][1])
                    iteration_done = True
                    break
            if not iteration_done:
                if self._tree._arb_eq(self._tree.payload, fwd[0][1]):
                    self.remove(self._tree.payload)
                elif self._tree._arb_gt(self._tree.payload, fwd[0][1]):
                    self.add(fwd[0][1])
                    self._tree = self._tree._splay(self._tree.right.minimum())
        for k, _ in fwd:
            try:
                items[k] = next(iters[k])
            except StopIteration:
                items.pop(k)

OrderedSet.symmetric_difference_update = symmetric_difference_update
del symmetric_difference_update

Als Operation haben wir `__ixor__` (`^=`).

In [58]:
def __ixor__(self, other):
    if isinstance(other, OrderedSet) or isinstance(other, OrderedFrozenset):
        self.symmetric_difference_update(other)
        return self
    else:
        raise TypeError("unsupported operand type(s) for ^=: " +
                        f"'{type(self).__name__}' and " +
                        f"'{type(other).__name__}'")

OrderedSet.__ixor__ = OrderedFrozenset.__ixor__ = __ixor__
del __ixor__